# Step 3: Upload Verified Links to Server

This notebook sends the verified CSV to a remote server for storage/processing.

In [ ]:
import requests
import pathlib
import json
from datetime import datetime

print('✅ Libraries imported')

## Configuration

In [ ]:
# File paths
csv_file = pathlib.Path('./links_table_with_status.csv')
server_endpoint = 'http://127.0.0.1:8774'
timeout_seconds = 30

print(f'CSV File: {csv_file}')
print(f'Endpoint: {server_endpoint}')

## Step 1: Validate CSV Exists

In [ ]:
if csv_file.exists():
    file_size_kb = csv_file.stat().st_size / 1024
    print(f'✅ CSV file found')
    print(f'   Size: {file_size_kb:.2f} KB')
else:
    print(f'❌ CSV file not found: {csv_file}')
    print('   Run Step 2 (Link Verification) first')

## Step 2: Read CSV Data

In [ ]:
with open(csv_file, 'r', encoding='utf-8') as f:
    csv_content = f.read()

lines = csv_content.split('\n')
print(f'✅ CSV loaded')
print(f'   Total lines: {len(lines)}')
print(f'   Data rows: {len(lines) - 1}')
print(f'\nFirst row (header):')
print(f'   {lines[0]}')
print(f'\nSample data row:')
if len(lines) > 1:
    print(f'   {lines[1][:100]}...')

## Step 3: Upload to Server

In [ ]:
print(f'📤 Uploading to {server_endpoint}...')
print(f'   Timeout: {timeout_seconds}s\n')

try:
    files = {
        'file': ('links_table_with_status.csv', csv_content, 'text/csv')
    }
    
    metadata = {
        'timestamp': datetime.now().isoformat(),
        'source': 'queens-cb2-link-grabber',
        'version': '1.0'
    }
    
    data = {
        'metadata': json.dumps(metadata)
    }
    
    response = requests.post(
        server_endpoint,
        files=files,
        data=data,
        timeout=timeout_seconds
    )
    
    print(f'📊 Response Status: {response.status_code}')
    
    if response.status_code in [200, 201, 202]:
        print('✅ Upload successful!')
        print(f'\nServer Response:')
        print(response.text[:500])
    else:
        print(f'⚠️ Server returned status {response.status_code}')
        print(f'Response: {response.text[:500]}')
        
except requests.exceptions.ConnectionError:
    print(f'❌ Connection Error')
    print(f'   Cannot reach {server_endpoint}')
    print(f'   Ensure server is running on port 8774')
except requests.exceptions.Timeout:
    print(f'❌ Timeout Error')
    print(f'   Server did not respond within {timeout_seconds}s')
except Exception as e:
    print(f'❌ Error: {e}')

## Summary

Upload workflow complete. Check server logs for confirmation.